# 1. Dirichlet fitting directly on MC2010

In [112]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

#Graphical parameters
mrkSz=5
pltSlcFactor=1000

import fitKelvinChain as fkc
import creepMC2010 as MC2010
import numpy as np
import matplotlib.pyplot as plt
cm = 1/2.54
fig, ax = plt.subplots(1, 1, figsize=(8*cm, 11*cm), constrained_layout=True)
fontSize = 8
plt.rc('font', size=fontSize)
plt.rc('axes', titlesize=fontSize)
plt.rc('legend',**{'fontsize':fontSize})

#Create EC2 compliances
#This is a concrete with personalized E-Modulus evolution by a three parameter law
costConcrete = MC2010.creepMC2010(250*250, 2*500, 'C50', '42.5N', 100, modulusCalculatedPerCode=False, considerTemperatureEffect=False, α_ThreeParModulus=36000, β_ThreeParModulus=1, 𝜏_ThreeParModulus=28)

#This is a standard Eurocode 2 concrete
#costConcrete = cEC2.creepEC2(250*250, 2*500, 'C50', '42.5N', 100, aggregateType='limestone',considerTemperatureEffect=False)
timeHistory=[2000]
temperatureHistory=[20]

#Set time scale of the simulation
numYearsOfCreep=10
numCreepPoints=10 #This value is used in the logarithmic scale, so it is hard to explain what is it. The higher the more creep points the creep curves will have
initialLoadingAge=3 #days
finalLoadingAge=90 #days
numOfLoadingAgesTested=4
initialStrain=[]
loading_age = np.geomspace(initialLoadingAge, finalLoadingAge,num=numOfLoadingAgesTested)
timeSpanBase = np.geomspace((1e-5),(365*numYearsOfCreep),num=numYearsOfCreep*(365*numCreepPoints)) 
#Initialize compliance vector
complianceSeries=[]
creepTimeSeries = [[] for age in loading_age]
J=[[] for ag in loading_age]
#Generate EC2 data to fit in DPL
color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loading_age))))

for enum, age in enumerate(loading_age):
    #Build the time span vector for this loading age
    
    timeSpan = timeSpanBase+age
    t0 = age #Just a dummy variable so everything looks beautiful inside the function call below

    fullCompliance = [(1/costConcrete.Eci_computeEvolution(t0))+(costConcrete.ϕ_compute(t,t0,timeHistory,temperatureHistory))/costConcrete.Eci for t in timeSpan]
    initialStrain=[1/costConcrete.Eci_computeEvolution(t0)]

    #Organizes vectors to be stored. We manually include "elastic" deformation, imediately after load application
    xaxis=[value for value in timeSpan]
    yaxis=fullCompliance
    complianceSeries.append([xaxis,yaxis])
    [creepTimeSeries[enum].append(element) for element in xaxis]
    creepTimeSeries[enum]=np.array(creepTimeSeries[enum])
    np.random.seed(12)
    #[J[enum].append(element+np.random.normal(np.random.normal(0,0.15*element,1)[0],0.15*element, 1)[0]) for element in yaxis]
    [J[enum].append(element) for element in yaxis]
    J[enum]=np.array(J[enum])

    #Visualize data being stored, if you wish
    c = next(color)
    ax.plot(xaxis[::pltSlcFactor]-age,np.array(yaxis)[::pltSlcFactor]*1e6,label="MC10-"+"{:.1f}".format(t0)+"d", marker='x',markersize=mrkSz,c=c)


#Build the ageingComplianceSeries to feed to kelvinChainModel object
ageingComplianceSeries = [[t_line, complianceSeriesData] for t_line, complianceSeriesData in zip(loading_age,complianceSeries)]

#Instantiate kelvinChainModel object
kelvinModel = fkc.kelvinChainModel(ageingComplianceSeries)

#Fit a Dirichlet series
fittingData = {'model':'raw', 't_line': loading_age, 't': creepTimeSeries, 'J': J}
kelvinModel.fitDirichletSeries(fittingData, loadingAgesInterval=[initialLoadingAge,finalLoadingAge, numOfLoadingAgesTested], graphVisualization=True, mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor, axisObjectToPlot=ax)

ax.set_xlabel("Days since loading (days)")
ax.set_ylabel("Compliance ($MPa^{-1}$)")
ax.set_xscale('log')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
          fancybox=True, columnspacing=0.2, ncol=2, fontsize=fontSize)
ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
fig.subplots_adjust(wspace=0.0, hspace=0, right=.95)
fig.tight_layout()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(array([  2.50448362, 379.10385444, 302.69924061,  82.58637134,
        17.31400083,  11.52676438,  10.11272318,   7.58290066,
         7.3144548 ,  10.0157235 ]), [inf, inf, inf, inf, inf, inf, inf, inf, inf, inf])
(array([   3.2766848 , 1332.37343395,  533.42179301,  379.6278071 ,
         84.71548338,   16.56022204,   11.28977612,    8.26385889,
          7.43391897,   10.0157235 ]), [inf, inf, inf, inf, inf, inf, inf, inf, inf, inf])
(array([   3.57463065, 2384.85546716, 1727.53923285,  492.81848698,
        416.38700327,   66.01489283,   15.68108185,    9.09729834,
          7.88191979,   10.0157235 ]), [inf, inf, inf, inf, inf, inf, inf, inf, inf, inf])


C:\Users\renan\AppData\Local\Temp\ipykernel_15764\732186859.py:87: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  fig.subplots_adjust(wspace=0.0, hspace=0, right=.95)
C:\Users\renan\AppData\Local\Temp\ipykernel_15764\732186859.py:88: UserWarning: The figure layout has changed to tight
  fig.tight_layout()


# 2. Dirichlet fitting through a modified DPL smoothing a EC2 compliance with artificial noise

In [120]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

import fitKelvinChain as fkc
import creepEC2 as cEC2
import numpy as np
import matplotlib.pyplot as plt

#Graphical parameters
mrkSz=5
pltSlcFactor=1000
cm = 1/2.54
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15*cm, 11*cm), constrained_layout=True)
fontSize = 8
plt.rc('font', size=fontSize)
plt.rc('axes', titlesize=fontSize)
plt.rc('legend',**{'fontsize':fontSize})

#Create EC2 compliances
#This is a concrete with personalized E-Modulus evolution by a three parameter law
costConcrete = cEC2.creepEC2(250*250, 2*500, 'C50', '42.5N', 100, modulusCalculatedPerCode=False, considerTemperatureEffect=False, α_ThreeParModulus=36000, β_ThreeParModulus=1, 𝜏_ThreeParModulus=28)

#This is a standard Eurocode 2 concrete
#costConcrete = cEC2.creepEC2(250*250, 2*500, 'C50', '42.5N', 100, aggregateType='limestone',considerTemperatureEffect=False)
timeHistory=[2000]
temperatureHistory=[20]

#Set time scale of the simulation
numYearsOfCreep=1
numCreepPoints=50 #This value is used in the logarithmic scale, so it is hard to explain what is it. The higher the more creep points the creep curves will have
initialLoadingAge=3 #days
finalLoadingAge=90 #days
numOfLoadingAgesTested=4
initialStrain=[]
loading_age = np.geomspace(initialLoadingAge, finalLoadingAge,num=numOfLoadingAgesTested)
timeSpanBase = np.geomspace((1e-5),(365*numYearsOfCreep),num=numYearsOfCreep*(365*numCreepPoints))
creepTimeSeries = [[] for age in loading_age]
J=[[] for ag in loading_age]

#Initialize compliance vector
complianceSeries = []
#Generate EC2 data to fit in DPL
color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loading_age))))
#Visualize data being stored, if you wish
for enum, age in enumerate(loading_age):
    #Build the time span vector for this loading age
    timeSpan = timeSpanBase+age
    t0 = age #Just a dummy variable so everything looks beautiful inside the function call below

    fullCompliance = [(1/costConcrete.Ecm_computeEvolution(t0))+(costConcrete.ϕ_compute(t,t0,timeHistory,temperatureHistory))/costConcrete.Ecm for t in timeSpan]
    initialStrain=[1/costConcrete.Ecm_computeEvolution(t0)]

    #Organizes vectors to be stored. We manually include "elastic" deformation, imediately after load application
    xaxis=np.array([value for value in timeSpan])
    yaxis=fullCompliance
    complianceSeries.append([xaxis,yaxis])

    np.random.seed(12)
    [creepTimeSeries[enum].append(element) for element in xaxis]
    creepTimeSeries[enum]=np.array(creepTimeSeries[enum])
    [J[enum].append(element+np.random.normal(0,0.01*element, 1)[0]) for element in yaxis]
    J[enum]=np.array(J[enum])

    
    #Visualize data being stored, if you wish
    
    c = next(color)
    #plt.plot(xaxis[::pltSlcFactor]-age,np.array(yaxis)[::pltSlcFactor]*1e6,label="EC2-"+"{:.1f}".format(t0)+"d", marker='x',markersize=mrkSz,c=c)
    ax1.plot(xaxis[::pltSlcFactor]-age,J[enum][::pltSlcFactor]*1e6,label="EC2-"+"{:.1f}".format(t0)+"d", marker='o',markersize=mrkSz,c=c)
    
ax1.set_xlabel("Days since loading (days)")
ax1.set_ylabel("Compliance ($MPa^{-1}$)")
ax1.set_ylim([25,70])
ax1.set_xscale('log')
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
         fancybox=True, columnspacing=0.5, ncol=1, fontsize=fontSize)
ax1.grid(which='minor', alpha=0.2)
ax1.grid(which='major', alpha=0.5)
fig.tight_layout()

#Build the ageingComplianceSeries to feed to kelvinChainModel object
ageingComplianceSeries = [[t_line, complianceSeriesData] for t_line, complianceSeriesData in zip(loading_age,complianceSeries)]

#Instantiate kelvinChainModel object
kelvinModel = fkc.kelvinChainModel(ageingComplianceSeries)

pltSlcFactor=1000

#Fit a classical DPL 
kelvinModel.fitDoublePowerLaw(graphVisualization=True, typeOfDPL='modified', mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor, axisObjectToPlot=ax2)

#Fit a Dirichlet series
fittingData = {'model':'raw', 't_line': loading_age, 't': creepTimeSeries, 'J': J}
kelvinModel.fitDirichletSeries(kelvinModel.modifiedDoublePowerLawModel, loadingAgesInterval=[initialLoadingAge,finalLoadingAge, numOfLoadingAgesTested], creepTimesInterval=[1e-5, 365*numYearsOfCreep, numYearsOfCreep*(365*numCreepPoints)], graphVisualization=True, mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor, axisObjectToPlot=ax2)
#kelvinModel.fitDirichletSeries(fittingData, loadingAgesInterval=[initialLoadingAge,finalLoadingAge, numOfLoadingAgesTested], graphVisualization=True, mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor, axisObjectToPlot=ax2)

ax2.set_xlabel("Days since loading (days)")
#ax2.set_ylabel("Compliance ($MPa^{-1}$)")
ax2.set_ylim([25,70])
ax2.set_xscale('log')
ax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
          fancybox=True, columnspacing=0.2, ncol=2, fontsize=fontSize)
ax2.grid(which='minor', alpha=0.2)
ax2.grid(which='major', alpha=0.5)
fig.subplots_adjust(wspace=0.0, hspace=0, right=.95)

fig.tight_layout()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\renan\AppData\Local\Temp\ipykernel_15764\2616917168.py:80: UserWarning: The figure layout has changed to tight
  fig.tight_layout()


(array([  1.67664496, 570.27002635, 275.2445274 , 145.22936903,
        68.96608944,  37.45645126,  17.06143836,  10.18666292,
         3.16389054]), [inf, inf, inf, inf, inf, inf, inf, inf, inf])
(array([  2.18189408, 708.02556558, 341.7334496 , 180.31149722,
        85.62575206,  46.50455286,  21.18285457,  12.64738608,
         3.92817015]), [inf, inf, inf, inf, inf, inf, inf, inf, inf])
(array([  2.37596893, 879.0604136 , 424.28363787, 223.86805537,
       106.30980068,  57.73834044,  26.2998535 ,  15.70252945,
         4.87707161]), [inf, inf, inf, inf, inf, inf, inf, inf, inf])


In [ ]:
print("Sum of Dirichlet series modulus:")
[print(sum(series)) for series in kelvinModel.dirichletSeriesModel['agingModulus']]
print("Dirichlet series modulus:")
print(kelvinModel.dirichletSeriesModel['agingModulus'])
print("Dirichlet series retardation times:")
print(kelvinModel.dirichletSeriesModel['retardationTimes'])
print("DPL E0 value:")
print(kelvinModel.modifiedDoublePowerLawModel['E0']/kelvinModel.modifiedDoublePowerLawModel['conversionFactor_compliance'])

In [ ]:
for enum, chain in  enumerate(kelvinModel.dirichletSeriesModel['agingModulus'][0]): 
    modulusSeries=[modulus[enum] for modulus in kelvinModel.dirichletSeriesModel['agingModulus']]
    plt.plot(kelvinModel.dirichletSeriesModel['t_line'], modulusSeries, label="Chain "+str(enum))
plt.legend()
plt.yscale('log')
plt.show()


# 3. See retardation spectrum

## Adjust a clean EC2 Kelvin chain

In [ ]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

import fitKelvinChain as fkc
import creepEC2 as cEC2
import numpy as np
import matplotlib.pyplot as plt

mrkSz=5

#Create EC2 compliances
#This is a concrete with personalized E-Modulus evolution by a three parameter law
costConcrete = cEC2.creepEC2(250*250, 2*500, 'C50', '42.5N', 100, modulusCalculatedPerCode=False, considerTemperatureEffect=False, α_ThreeParModulus=36000, β_ThreeParModulus=1, 𝜏_ThreeParModulus=28)

#This is a standard Eurocode 2 concrete
#costConcrete = cEC2.creepEC2(250*250, 2*500, 'C50', '42.5N', 100, aggregateType='limestone',considerTemperatureEffect=False)
timeHistory=[2000]
temperatureHistory=[20]

#Set time scale of the simulation
numYearsOfCreep=1
numCreepPoints=50 #This value is used in the logarithmic scale, so it is hard to explain what is it. The higher the more creep points the creep curves will have
initialLoadingAge=3 #days
finalLoadingAge=90 #days
numOfLoadingAgesTested=15
initialStrain=[]
loading_age = np.geomspace(initialLoadingAge, finalLoadingAge,num=numOfLoadingAgesTested)
timeSpanBase = np.geomspace((1e-5),(365*numYearsOfCreep),num=numYearsOfCreep*(365*numCreepPoints))
creepTimeSeries = [[] for age in loading_age]
J=[[] for ag in loading_age]

#Initialize compliance vector
complianceSeries = []
#Generate EC2 data to fit in DPL
color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loading_age))))
#Visualize data being stored, if you wish
for enum, age in enumerate(loading_age):
    #Build the time span vector for this loading age
    timeSpan = timeSpanBase+age
    t0 = age #Just a dummy variable so everything looks beautiful inside the function call below

    fullCompliance = [(1/costConcrete.Ecm_computeEvolution(t0))+(costConcrete.ϕ_compute(t,t0,timeHistory,temperatureHistory))/costConcrete.Ecm for t in timeSpan]
    initialStrain=[1/costConcrete.Ecm_computeEvolution(t0)]

    #Organizes vectors to be stored. We manually include "elastic" deformation, imediately after load application
    xaxis=np.array([value for value in timeSpan])
    yaxis=fullCompliance
    complianceSeries.append([xaxis,yaxis])
    [creepTimeSeries[enum].append(element) for element in xaxis]
    creepTimeSeries[enum]=np.array(creepTimeSeries[enum])
    [J[enum].append(element) for element in yaxis]
    J[enum]=np.array(J[enum])

    #Visualize data being stored, if you wish
    c = next(color)
    #plt.plot(xaxis[::pltSlcFactor]-age,np.array(yaxis)[::pltSlcFactor]*1e6,label="EC2-"+"{:.1f}".format(t0)+"d", marker='x',markersize=mrkSz,c=c)
    plt.plot(xaxis[::pltSlcFactor]-age,J[enum][::pltSlcFactor]*1e6,label="EC2-"+"{:.1f}".format(t0)+"d", marker='o',markersize=mrkSz,c=c)
    plt.xlabel("Days since loading (days)")
    plt.ylabel("Compliance ($MPa^{-1}$)")
    

'''
plt.legend()
plt.plot()
'''

#Build the ageingComplianceSeries to feed to kelvinChainModel object
ageingComplianceSeries = [[t_line, complianceSeriesData] for t_line, complianceSeriesData in zip(loading_age,complianceSeries)]

#Instantiate kelvinChainModel object
kelvinModel_EC2clean = fkc.kelvinChainModel(ageingComplianceSeries)

pltSlcFactor=1000

#Fit a Dirichlet series
fittingData = {'model':'raw', 't_line': loading_age, 't': creepTimeSeries, 'J': J}

kelvinModel_EC2clean.fitDirichletSeries(fittingData, loadingAgesInterval=[initialLoadingAge,finalLoadingAge, numOfLoadingAgesTested], retardationTimesFactor=0.5, graphVisualization= True, mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor)

## Plot the retardation spectrum of EC2

In [ ]:
retardationTimes=kelvinModel_EC2clean.dirichletSeriesModel['retardationTimes']
agingModulus=kelvinModel_EC2clean.dirichletSeriesModel['agingModulus']
loadingAges=kelvinModel_EC2clean.dirichletSeriesModel['t_line']

color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loadingAges))))
for enum, series in enumerate(agingModulus):
    c = next(color)
    plt.plot(retardationTimes, series, label="{:.1f}".format(loadingAges[enum]), c=c)
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

## Adjust a clean MC2010 Kelvin Chain

In [ ]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

#Graphical parameters
mrkSz=5
pltSlcFactor=1000

import fitKelvinChain as fkc
import creepMC2010 as MC2010
import numpy as np
import matplotlib.pyplot as plt

#Create EC2 compliances
#This is a concrete with personalized E-Modulus evolution by a three parameter law
costConcrete = MC2010.creepMC2010(250*250, 2*500, 'C50', '42.5N', 60, modulusCalculatedPerCode=False, considerTemperatureEffect=False, α_ThreeParModulus=36000, β_ThreeParModulus=1, 𝜏_ThreeParModulus=28)

#This is a standard Eurocode 2 concrete
#costConcrete = cEC2.creepEC2(250*250, 2*500, 'C50', '42.5N', 100, aggregateType='limestone',considerTemperatureEffect=False)
timeHistory=[2000]
temperatureHistory=[20]

#Set time scale of the simulation
numYearsOfCreep=10
numCreepPoints=10 #This value is used in the logarithmic scale, so it is hard to explain what is it. The higher the more creep points the creep curves will have
initialLoadingAge=3 #days
finalLoadingAge=90 #days
numOfLoadingAgesTested=3
initialStrain=[]
loading_age = np.geomspace(initialLoadingAge, finalLoadingAge,num=numOfLoadingAgesTested)
timeSpanBase = np.geomspace((1e-5),(365*numYearsOfCreep),num=numYearsOfCreep*(365*numCreepPoints)) 
#Initialize compliance vector
complianceSeries=[]
creepTimeSeries = [[] for age in loading_age]
J=[[] for ag in loading_age]
#Generate EC2 data to fit in DPL
color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loading_age))))

for enum, age in enumerate(loading_age):
    #Build the time span vector for this loading age
    
    timeSpan = timeSpanBase+age
    t0 = age #Just a dummy variable so everything looks beautiful inside the function call below

    fullCompliance = [(costConcrete.ϕ_compute(t,t0,timeHistory,temperatureHistory)) for t in timeSpan]
    initialStrain=[1/costConcrete.Eci_computeEvolution(t0)]

    #Organizes vectors to be stored. We manually include "elastic" deformation, imediately after load application
    xaxis=[value for value in timeSpan]
    yaxis=fullCompliance
    complianceSeries.append([xaxis,yaxis])
    [creepTimeSeries[enum].append(element) for element in xaxis]
    creepTimeSeries[enum]=np.array(creepTimeSeries[enum])
    [J[enum].append(element) for element in yaxis]
    J[enum]=np.array(J[enum])

    #Visualize data being stored, if you wish
    c = next(color)
    plt.plot(xaxis[::pltSlcFactor]-age,np.array(yaxis)[::pltSlcFactor]*1e6,label="MC20-"+"{:.1f}".format(t0)+"d", marker='x',markersize=mrkSz,c=c)
    plt.xlabel("Days since loading (days)")
    plt.ylabel("Compliance ($MPa^{-1}$)")

#Build the ageingComplianceSeries to feed to kelvinChainModel object
ageingComplianceSeries = [[t_line, complianceSeriesData] for t_line, complianceSeriesData in zip(loading_age,complianceSeries)]

#Instantiate kelvinChainModel object
kelvinModel_MC2010clean = fkc.kelvinChainModel(ageingComplianceSeries)

#Fit a Dirichlet series
fittingData = {'model':'raw', 't_line': loading_age, 't': creepTimeSeries, 'J': J}
kelvinModel_MC2010clean.fitDirichletSeries(fittingData, loadingAgesInterval=[initialLoadingAge,finalLoadingAge, numOfLoadingAgesTested], retardationTimesFactor=0.5, graphVisualization=True, mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor)

In [ ]:
retardationTimes=kelvinModel_MC2010clean.dirichletSeriesModel['retardationTimes']
agingModulus=kelvinModel_MC2010clean.dirichletSeriesModel['agingModulus']
loadingAges=kelvinModel_MC2010clean.dirichletSeriesModel['t_line']

color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loadingAges))))
for enum, series in enumerate(agingModulus):
    c = next(color)
    plt.plot(retardationTimes, series, label="{:.1f}".format(loadingAges[enum]), c=c)
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()